<a href="https://colab.research.google.com/github/gsukisubramaniam/Projects/blob/master/Nifty500_Scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import os

In [3]:
# To ignore the warnings for pandas operations
pd.options.mode.chained_assignment = None

Loading the datasets

In [4]:
nifty_500 = pd.read_csv('drive/MyDrive/trading/ind_nifty500list.csv')
 
# Nifty 500 list
 
nifty_500_symbols = np.array(nifty_500['Symbol'])
print (len(nifty_500_symbols))
print (type(nifty_500_symbols))

501
<class 'numpy.ndarray'>


Master Datasets

In [5]:
# RSI updated datasources
df = pd.read_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_df_RSI.csv',infer_datetime_format = True, parse_dates=['TIMESTAMP'], encoding='utf-8')
df_w = pd.read_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_weekly_RSI.csv',infer_datetime_format = True, parse_dates=['TIMESTAMP'], encoding='utf-8')
df_m = pd.read_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_monthly_RSI.csv',infer_datetime_format = True, parse_dates=['TIMESTAMP'], encoding='utf-8')

In [6]:
# Ichmioku updated datasources
df_ichimoku_w = pd.read_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_weekly_ich.csv',infer_datetime_format = True, parse_dates=['TIMESTAMP'], encoding='utf-8')
df_ichimoku_d = pd.read_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_daily_ich.csv',infer_datetime_format = True, parse_dates=['TIMESTAMP'], encoding='utf-8')

In [7]:
# Swing High & Swing Low Datasources
df_swing = pd.read_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_df_swing.csv',infer_datetime_format = True, parse_dates=['TIMESTAMP'], encoding='utf-8')

# Screeners

Screener 1 : RSI Screener

In [8]:
rsi_screen_stocks = []
for symbol in nifty_500_symbols:
    m_check = df_m.loc[df_m['SYMBOL']==symbol].iat[-1, 16]
    w_check = df_w.loc[df_w['SYMBOL']==symbol].iat[-1, 16]
    if df.loc[df['SYMBOL']==symbol]['RSI_EMA'][-15:-1].values.max() < 60:
        d_last14days = 1
    else:
        d_last14days = 0
    d_latestRSI = df.loc[df['SYMBOL']==symbol].iat[-1,16]
    if ((m_check > 60) & (w_check > 60) & (d_last14days==1) & (d_latestRSI > 60)):
        rsi_screen_stocks.append(symbol)
    else:
        pass
 
print(rsi_screen_stocks)

['GUJGASLTD', 'SOBHA', 'TATACONSUM', 'VOLTAS', 'ECLERX']


Screener 2: ICHIMOKU Screener

Ichimoku Screener Logic

1. Weekly Senkou_A > Weekly Senkou_B 
2. Daily Senkou_A crossed above Daily Senkou_B

In [9]:
ichimoku_screen_stocks = []
for symbol in nifty_500_symbols:
    temp_d = df_ichimoku_d.loc[df_ichimoku_d['SYMBOL']==symbol]
    temp_d.sort_values(by='TIMESTAMP',inplace=True)
    temp_d.reset_index(drop=True,inplace=True)
 
    temp_w = df_ichimoku_w.loc[df_ichimoku_w['SYMBOL']==symbol]
    temp_w.sort_values(by='TIMESTAMP',inplace=True)
    temp_w.reset_index(drop=True,inplace=True)
    
    if (temp_d.iat[-1, 9] >= temp_d.iat[-1, 10]) & (temp_d.iat[-2, 9] < temp_d.iat[-2, 10]) & (temp_w.iat[-1,9]>temp_w.iat[-1,10]):
        ichimoku_screen_stocks.append(symbol)
    else:
        pass
 
print (ichimoku_screen_stocks)

['ADANIPORTS', 'APOLLOTYRE', 'DCBBANK', 'EXIDEIND', 'GUJGASLTD', 'HDFCAMC', 'IDFCFIRSTB', 'IRB', 'JUSTDIAL', 'KAJARIACER', 'NAVNETEDUL', 'SUNDARMFIN', 'VOLTAS']


Screener 3: Swing High break Screener

In [10]:
swing_screen_stocks = []
for symbol in nifty_500_symbols:
    temp_sw_d = df_swing.loc[df_swing['SYMBOL']==symbol]
    temp_sw_d.sort_values(by='TIMESTAMP', inplace=True)
    temp_sw_d.reset_index(drop = True, inplace=True)
    if (temp_sw_d.iat[-1, 5] >= temp_sw_d.iat[-1, 7]) & (temp_sw_d.iat[-2, 5] < temp_sw_d.iat[-2, 7]):
        swing_screen_stocks.append(symbol)
    else:
        pass

In [11]:
print (pd.Series(swing_screen_stocks))

0         AUBANK
1      AEGISCHEM
2     APOLLOTYRE
3     BAJAJ-AUTO
4     BAJFINANCE
5     BALKRISIND
6       MAHABANK
7     BHARATFORG
8         BIOCON
9         CRISIL
10         CANBK
11     CENTRALBK
12         CIPLA
13    LALPATHLAB
14      EMAMILTD
15           FSL
16        GESHIP
17      HDFCLIFE
18    ICICIPRULI
19    IDFCFIRSTB
20           IOB
21    INDUSINDBK
22    JBCHEPHARM
23           KEI
24    KAJARIACER
25         LUPIN
26           MRF
27        MAHLOG
28        MARICO
29    MOTHERSUMI
30      NILKAMAL
31           OIL
32         PIIND
33           PVR
34       RBLBANK
35        RECLTD
36       RAYMOND
37        SANOFI
38       SIEMENS
39      SUMICHEM
40        SUZLON
41        TCIEXP
42           UPL
43        VARROC
44        VOLTAS
dtype: object


Stocks in common 

In [12]:
# rsi_scr = rsi_screen_stocks
# ich_scr = ichimoku_screen_stocks
# sw_scr = swing_screen_stocks
 
# sto_rsi_swing = list(set(rsi_scr).intersection(sw_scr))
# print(f'Common stocks from RSI screener & Swing High Break screener: ', sto_rsi_swing)
 
# sto_ich_swing = list(set(ich_scr).intersection(sw_scr))
# print(f'Common stocks from Ichimoku screener & Swing High Break screener: ', sto_ich_swing)

4th Screener: Swing High Break && Ichimoku (Daily/Weekly Senkou A) > (Daily/Weekly Senkou B)

In [13]:
# sw_ich_screen_stocks = []
# for symbol in nifty_500_symbols:
#     temp_d = df_ichimoku_d.loc[df_ichimoku_d['SYMBOL']==symbol]
#     temp_w = df_ichimoku_w.loc[df_ichimoku_w['SYMBOL']==symbol]
#     temp_sw_d = df_swing.loc[df_swing['SYMBOL']==symbol]
 
#     temp_d.sort_values(by='TIMESTAMP', inplace=True)
#     temp_w.sort_values(by='TIMESTAMP', inplace=True)
#     temp_sw_d.sort_values(by='TIMESTAMP', inplace=True)
 
#     temp_d.reset_index(drop=True, inplace=True)
#     temp_w.reset_index(drop=True, inplace=True)
#     temp_sw_d.reset_index(drop=True, inplace=True)
 
#     if (temp_d.iat[-1, 9] > temp_d.iat[-1, 10]) & (temp_w.iat[-1,9]>temp_w.iat[-1,10]) & (temp_sw_d.iat[-1, 5] >= temp_sw_d.iat[-1, 7]) & (temp_sw_d.iat[-2, 5] < temp_sw_d.iat[-2, 7]):
#         sw_ich_screen_stocks.append(symbol)
#     else:
#         pass
# print (pd.Series(sw_ich_screen_stocks))

In [14]:
print ('RSI Screener Stocks: ')
print (rsi_screen_stocks)
print ('------')
print ('ICHIMOKU Screener Stocks:')
print (pd.Series(ichimoku_screen_stocks))
print ('------')
print ('Swing High break screener stocks:')
print (pd.Series(swing_screen_stocks))
# print ('------')
# print ('Swing High Break && Ichimoku (Daily/Weekly Senkou A > Senkou B: ')
# print (pd.Series(sw_ich_screen_stocks))
# print ('------')

RSI Screener Stocks: 
['GUJGASLTD', 'SOBHA', 'TATACONSUM', 'VOLTAS', 'ECLERX']
------
ICHIMOKU Screener Stocks:
0     ADANIPORTS
1     APOLLOTYRE
2        DCBBANK
3       EXIDEIND
4      GUJGASLTD
5        HDFCAMC
6     IDFCFIRSTB
7            IRB
8       JUSTDIAL
9     KAJARIACER
10    NAVNETEDUL
11    SUNDARMFIN
12        VOLTAS
dtype: object
------
Swing High break screener stocks:
0         AUBANK
1      AEGISCHEM
2     APOLLOTYRE
3     BAJAJ-AUTO
4     BAJFINANCE
5     BALKRISIND
6       MAHABANK
7     BHARATFORG
8         BIOCON
9         CRISIL
10         CANBK
11     CENTRALBK
12         CIPLA
13    LALPATHLAB
14      EMAMILTD
15           FSL
16        GESHIP
17      HDFCLIFE
18    ICICIPRULI
19    IDFCFIRSTB
20           IOB
21    INDUSINDBK
22    JBCHEPHARM
23           KEI
24    KAJARIACER
25         LUPIN
26           MRF
27        MAHLOG
28        MARICO
29    MOTHERSUMI
30      NILKAMAL
31           OIL
32         PIIND
33           PVR
34       RBLBANK
35        RECLTD


# WatchList & Calls

**Ichimoku Watchlist & Calls**



1. Append the latest Ichimoku Screener stocks.
2. Either weekly or Daily Senkou A less than Senkou B to be removed
3. Swing High Break stocks to be copied to another file - "Date_Ichimoku_SwingHighBreak.csv"List item






In [15]:
# Loading the latest Ichimoku watchlist, calls, and expunged datasets for updation

ich_watchlist = pd.read_csv('drive/MyDrive/trading/WatchList_Calls/Ichimoku/ichimoku_watchlist.csv',infer_datetime_format = True, parse_dates=['TIMESTAMP'], encoding='utf-8')
ich_expunged = pd.read_csv('drive/MyDrive/trading/WatchList_Calls/Ichimoku/ichimoku_expunged_stocks.csv',infer_datetime_format = True, parse_dates=['TIMESTAMP'], encoding = 'utf-8')
ich_calls = pd.read_csv('drive/MyDrive/trading/WatchList_Calls/Ichimoku/ichimoku_calls.csv',infer_datetime_format = True, parse_dates=['TIMESTAMP'], encoding = 'utf-8')


In [16]:
# Today's Ichimoku Screener stocks as a dataframe
output = pd.DataFrame(columns = ['TIMESTAMP', 'SYMBOL'])
output['SYMBOL'] = pd.Series(ichimoku_screen_stocks)
output['TIMESTAMP'] = df_ichimoku_d['TIMESTAMP'].max()
output.head()

,TIMESTAMP,SYMBOL
0,2021-06-02,ADANIPORTS
1,2021-06-02,APOLLOTYRE
2,2021-06-02,DCBBANK
3,2021-06-02,EXIDEIND
4,2021-06-02,GUJGASLTD


In [17]:
watchlist_upd = pd.concat([ich_watchlist, output])
watchlist_upd.reset_index(drop=True,inplace=True)

watchlist_upd.tail(10)
# watchlist_upd = ich_watchlist.copy()

,TIMESTAMP,SYMBOL
88,2021-06-02,EXIDEIND
89,2021-06-02,GUJGASLTD
90,2021-06-02,HDFCAMC
91,2021-06-02,IDFCFIRSTB
92,2021-06-02,IRB
93,2021-06-02,JUSTDIAL
94,2021-06-02,KAJARIACER
95,2021-06-02,NAVNETEDUL
96,2021-06-02,SUNDARMFIN
97,2021-06-02,VOLTAS


In [18]:
ichimoku_rm_from_watchlist = []
ichimoku_swingHigh_break = []
for symbol in np.array(watchlist_upd['SYMBOL']):
    temp_d = df_ichimoku_d.loc[df_ichimoku_d['SYMBOL']==symbol]
    temp_d.sort_values(by='TIMESTAMP',inplace=True)
    temp_d.reset_index(drop=True,inplace=True)

    temp_w = df_ichimoku_w.loc[df_ichimoku_w['SYMBOL']==symbol]
    temp_w.sort_values(by='TIMESTAMP',inplace=True)
    temp_w.reset_index(drop=True,inplace=True)
    
    temp_sw_d = df_swing.loc[df_swing['SYMBOL']==symbol]
    temp_sw_d.sort_values(by='TIMESTAMP', inplace=True)
    temp_sw_d.reset_index(drop=True, inplace=True)

    if (temp_d.iat[-1, 9] < temp_d.iat[-1, 10]) or (temp_w.iat[-1,9] < temp_w.iat[-1,10]):
        ichimoku_rm_from_watchlist.append(symbol)
    else:
        pass
    if (temp_sw_d.iat[-1, 5] >= temp_sw_d.iat[-1, 7]) & (temp_sw_d.iat[-2, 5] < temp_sw_d.iat[-2, 7]):
        ichimoku_swingHigh_break.append(symbol)
    else:
        pass


print ('Stocks removed from Ichimoku Watchlist: ', ichimoku_rm_from_watchlist)
print ('Stocks identified as Ichimoku calls: ', ichimoku_swingHigh_break)

Stocks removed from Ichimoku Watchlist:  []
Stocks identified as Ichimoku calls:  ['BALKRISIND', 'SUZLON', 'IOB', 'CRISIL', 'CENTRALBK', 'INDUSINDBK', 'RAYMOND', 'APOLLOTYRE', 'IDFCFIRSTB', 'KAJARIACER', 'VOLTAS']


In [19]:
ichimoku_swingHigh_break

['BALKRISIND',
 'SUZLON',
 'IOB',
 'CRISIL',
 'CENTRALBK',
 'INDUSINDBK',
 'RAYMOND',
 'APOLLOTYRE',
 'IDFCFIRSTB',
 'KAJARIACER',
 'VOLTAS']

In [20]:
# Updating the Ichimoku Watchlist
watchlist_final = watchlist_upd[~watchlist_upd.SYMBOL.isin(ichimoku_rm_from_watchlist)]
watchlist_final = watchlist_final[~watchlist_final.SYMBOL.isin(ichimoku_swingHigh_break)]
watchlist_final.drop_duplicates(inplace=True)
watchlist_final.to_csv('drive/MyDrive/trading/WatchList_Calls/Ichimoku/ichimoku_watchlist.csv', index=False, encoding = 'utf-8')

# List of stocks expelled from the watchlist because of the Rule 1 break (Daily interval Senkou A < Senkou B (or) Weekly interval Senkou A < Senkou B )

try:
    expunged_records = watchlist_upd[watchlist_upd.SYMBOL.isin(ichimoku_rm_from_watchlist)]
    ich_expunged_upd = pd.concat([ich_expunged,expunged_records]) 
    ich_expunged_upd.sort_values(by='TIMESTAMP', inplace = True)
    ich_expunged_upd.drop_duplicates(inplace=True)
    ich_expunged_upd.reset_index(drop=True,inplace=True)
    ich_expunged_upd.to_csv('drive/MyDrive/trading/WatchList_Calls/Ichimoku/ichimoku_expunged_stocks.csv', index=False, encoding = 'utf-8')
    print ('Expunged stocks stored in csv')

    # Updating the Ichimoku Calls
    i_calls = watchlist_upd[watchlist_upd.SYMBOL.isin(ichimoku_swingHigh_break)]
    i_calls['TIMESTAMP'] = df_ichimoku_d['TIMESTAMP'].max()
    ich_calls_upd = pd.concat([ich_calls,i_calls]) 
    ich_calls_upd.sort_values(by='TIMESTAMP', inplace = True)
    ich_calls_upd.drop_duplicates(inplace=True)
    ich_calls_upd.reset_index(drop=True,inplace=True)
    ich_calls_upd.to_csv('drive/MyDrive/trading/WatchList_Calls/Ichimoku/ichimoku_calls.csv', index=False, encoding = 'utf-8')
    print ('Ichimoku Calls stored in csv')

except:
    print ('Failed. Check the code and file path.')

Expunged stocks stored in csv
Ichimoku Calls stored in csv


In [21]:
ich_calls_upd[-20:]

,TIMESTAMP,SYMBOL
142,2021-05-31,SJVN
143,2021-05-31,MGL
144,2021-05-31,WELCORP
145,2021-05-31,TRENT
146,2021-05-31,VRLLOG
147,2021-06-01,HDFC
148,2021-06-01,ONGC
149,2021-06-01,NH
150,2021-06-01,VIPIND
151,2021-06-02,IDFCFIRSTB


**RSI Watchlist & Calls** 

In [22]:
rsi_watchlist = pd.read_csv('drive/MyDrive/trading/WatchList_Calls/RSI/rsi_watchlist.csv',infer_datetime_format = True, parse_dates=['TIMESTAMP'], encoding = 'utf-8')
rsi_expunged = pd.read_csv('drive/MyDrive/trading/WatchList_Calls/RSI/rsi_expunged_stocks.csv',infer_datetime_format = True, parse_dates=['TIMESTAMP'], encoding = 'utf-8')
rsi_calls = pd.read_csv('drive/MyDrive/trading/WatchList_Calls/RSI/rsi_calls.csv',infer_datetime_format = True, parse_dates=['TIMESTAMP'], encoding = 'utf-8')


In [23]:
rsi_watchlist

,TIMESTAMP,SYMBOL
0,2021-05-12,CHAMBLFERT
1,2021-05-18,BSOFT
2,2021-05-19,JKLAKSHMI
3,2021-05-24,SOLARINDS
4,2021-05-28,JUSTDIAL
5,2021-05-28,SRF


Adding RSI screened stocks to the RSI Watchlist

In [24]:
rsi_temp = pd.DataFrame(columns = ['TIMESTAMP', 'SYMBOL'])
rsi_temp['SYMBOL'] = pd.Series(rsi_screen_stocks)
rsi_temp['TIMESTAMP'] = df['TIMESTAMP'].max()
# rsi_temp.head()

rsi_watchlist_upd = pd.concat([rsi_watchlist, rsi_temp])
rsi_watchlist_upd.drop_duplicates(inplace=True)
rsi_watchlist_upd.reset_index(drop=True,inplace=True)

rsi_watchlist_upd

,TIMESTAMP,SYMBOL
0,2021-05-12,CHAMBLFERT
1,2021-05-18,BSOFT
2,2021-05-19,JKLAKSHMI
3,2021-05-24,SOLARINDS
4,2021-05-28,JUSTDIAL
5,2021-05-28,SRF
6,2021-06-02,GUJGASLTD
7,2021-06-02,SOBHA
8,2021-06-02,TATACONSUM
9,2021-06-02,VOLTAS


In [25]:
np.array(rsi_watchlist_upd['SYMBOL'])

array(['CHAMBLFERT', 'BSOFT', 'JKLAKSHMI', 'SOLARINDS', 'JUSTDIAL', 'SRF',
       'GUJGASLTD', 'SOBHA', 'TATACONSUM', 'VOLTAS', 'ECLERX'],
      dtype=object)

In [26]:
rsi_rm_from_watchlist = []
rsi_swingHigh_break = []
for symbol in np.array(rsi_watchlist_upd['SYMBOL']):
    
    temp_d = df.loc[df['SYMBOL']==symbol]
    temp_d.sort_values(by='TIMESTAMP',inplace=True)
    temp_d.reset_index(drop=True,inplace=True)

    temp_w = df_w.loc[df_w['SYMBOL']==symbol]
    temp_w.sort_values(by='TIMESTAMP',inplace=True)
    temp_w.reset_index(drop=True,inplace=True)

    temp_m = df_m.loc[df_m['SYMBOL']==symbol]
    temp_m.sort_values(by='TIMESTAMP',inplace=True)
    temp_m.reset_index(drop=True,inplace=True)
    
    temp_sw_d = df_swing.loc[df_swing['SYMBOL']==symbol]
    temp_sw_d.sort_values(by='TIMESTAMP', inplace=True)
    temp_sw_d.reset_index(drop=True, inplace=True)
    
    


    # ------
    m_check = temp_m.loc[temp_m['SYMBOL']==symbol].iat[-1, 16]
    w_check = temp_w.loc[temp_w['SYMBOL']==symbol].iat[-1, 16]
    if temp_d.loc[temp_d['SYMBOL']==symbol]['RSI_EMA'][-15:-1].values.max() < 60:
        d_last14days = 1
    else:
        d_last14days = 0
    d_latestRSI = df.loc[df['SYMBOL']==symbol].iat[-1,16]
    if ((m_check > 60) & (w_check > 60) & (d_last14days==1) & (d_latestRSI > 60)):
        if (temp_sw_d.iat[-1, 5] >= temp_sw_d.iat[-1, 7]) & (temp_sw_d.iat[-2, 5] < temp_sw_d.iat[-2, 7]):
            rsi_swingHigh_break.append(symbol)
        else:
            pass
    elif ((m_check < 60) or (w_check < 60) or (d_latestRSI < 60)):
        rsi_rm_from_watchlist.append(symbol)
    else:
        pass



print ('Stocks to be removed from RSI Watchlist: ',rsi_rm_from_watchlist)
print ('Stocks identified as RSI calls: ',rsi_swingHigh_break)

Stocks to be removed from RSI Watchlist:  []
Stocks identified as RSI calls:  ['VOLTAS']


In [27]:
# Updating the RSI Watchlist
rsi_watchlist_final = rsi_watchlist_upd[~rsi_watchlist_upd.SYMBOL.isin(rsi_rm_from_watchlist)]
rsi_watchlist_final = rsi_watchlist_final[~rsi_watchlist_final.SYMBOL.isin(rsi_swingHigh_break)]
rsi_watchlist_final.drop_duplicates(inplace=True)
rsi_watchlist_final.to_csv('drive/MyDrive/trading/WatchList_Calls/RSI/rsi_watchlist.csv', index=False, encoding = 'utf-8')

# List of stocks expelled from the watchlist because of the Rule 1 break (Weekly or Monthly or Daily RSI less than 60)

try:
    expunged_records = rsi_watchlist_upd[rsi_watchlist_upd.SYMBOL.isin(rsi_rm_from_watchlist)]
    rsi_expunged_upd = pd.concat([rsi_expunged,expunged_records]) 
    rsi_expunged_upd.sort_values(by='TIMESTAMP', inplace = True)
    rsi_expunged_upd.drop_duplicates(inplace=True)
    rsi_expunged_upd.reset_index(drop=True,inplace=True)
    rsi_expunged_upd.to_csv('drive/MyDrive/trading/WatchList_Calls/RSI/rsi_expunged_stocks.csv', index=False, encoding = 'utf-8')
    print ('Expunged stocks stored in csv')

    # Updating the RSI Calls
    r_calls = rsi_watchlist_upd[rsi_watchlist_upd.SYMBOL.isin(rsi_swingHigh_break)]
    r_calls['TIMESTAMP'] = df['TIMESTAMP'].max()
    rsi_calls_upd = pd.concat([rsi_calls,r_calls]) 
    rsi_calls_upd.sort_values(by='TIMESTAMP', inplace = True)
    rsi_calls_upd.drop_duplicates(inplace=True)
    rsi_calls_upd.reset_index(drop=True,inplace=True)
    rsi_calls_upd.to_csv('drive/MyDrive/trading/WatchList_Calls/RSI/rsi_calls.csv', index=False, encoding = 'utf-8')
    print ('RSI Calls stored in csv')

except:
    print ('Failed. Check the code and file path.')

Expunged stocks stored in csv
RSI Calls stored in csv


In [28]:
rsi_calls_upd

,TIMESTAMP,SYMBOL
0,2021-04-22,SUVENPHAR
1,2021-04-23,TTKPRESTIG
2,2021-04-27,SUDARSCHEM
3,2021-04-27,SUPREMEIND
4,2021-04-28,AEGISCHEM
...,...,...
56,2021-05-31,THYROCARE
57,2021-05-31,NAM-INDIA
58,2021-05-31,RATNAMANI
59,2021-06-01,ADANIPORTS


---------------------------------------- End of Screener --------------------------

Calls - Analysis

Loading the latest datasets

In [29]:
ich_calls = pd.read_csv('drive/MyDrive/trading/WatchList_Calls/Ichimoku/ichimoku_calls.csv',infer_datetime_format = True, parse_dates=['TIMESTAMP'], encoding = 'utf-8')
rsi_calls = pd.read_csv('drive/MyDrive/trading/WatchList_Calls/RSI/rsi_calls.csv',infer_datetime_format = True, parse_dates=['TIMESTAMP'], encoding = 'utf-8')
df_swing = pd.read_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_df_swing.csv',infer_datetime_format = True, parse_dates=['TIMESTAMP'], encoding='utf-8')

ich_calls[:10]

,TIMESTAMP,SYMBOL
0,2021-04-13,MARICO
1,2021-04-16,JSLHISAR
2,2021-04-16,WIPRO
3,2021-04-16,STAR
4,2021-04-16,AUROPHARMA
5,2021-04-16,KSCL
6,2021-04-16,SWARAJENG
7,2021-04-20,WOCKPHARMA
8,2021-04-20,ICICIPRULI
9,2021-04-20,DRREDDY


In [30]:
df_swing.tail()

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,TIMESTAMP,swing_high,swing_low
292154,ECLERX,EQ,1100.0,1201.00,1099.00,1179.65,2021-05-27,1223.9,1099.0
292155,ECLERX,EQ,1200.0,1245.00,1180.05,1236.35,2021-05-28,1223.9,1099.0
292156,ECLERX,EQ,1247.7,1265.50,1226.20,1260.15,2021-05-31,1223.9,1099.0
292157,ECLERX,EQ,1260.5,1266.75,1231.05,1254.05,2021-06-01,1223.9,1099.0
292158,ECLERX,EQ,1250.0,1324.00,1242.00,1265.85,2021-06-02,1223.9,1099.0


In [31]:
ich_calls[-20:]

,TIMESTAMP,SYMBOL
142,2021-05-31,SJVN
143,2021-05-31,MGL
144,2021-05-31,WELCORP
145,2021-05-31,TRENT
146,2021-05-31,VRLLOG
147,2021-06-01,HDFC
148,2021-06-01,ONGC
149,2021-06-01,NH
150,2021-06-01,VIPIND
151,2021-06-02,IDFCFIRSTB


In [32]:
calls = ich_calls.copy()
calls_j = pd.merge(calls, df_swing, on =['SYMBOL','TIMESTAMP'], how ='left')[['TIMESTAMP','SYMBOL','CLOSE']]
calls_j[:10]

,TIMESTAMP,SYMBOL,CLOSE
0,2021-04-13,MARICO,421.30
1,2021-04-16,JSLHISAR,140.80
2,2021-04-16,WIPRO,469.20
3,2021-04-16,STAR,922.15
4,2021-04-16,AUROPHARMA,953.00
5,2021-04-16,KSCL,569.80
6,2021-04-16,SWARAJENG,1374.00
7,2021-04-20,WOCKPHARMA,506.25
8,2021-04-20,ICICIPRULI,477.95
9,2021-04-20,DRREDDY,5156.75


In [33]:
calls_j.tail(12)

,TIMESTAMP,SYMBOL,CLOSE
150,2021-06-01,VIPIND,382.75
151,2021-06-02,IDFCFIRSTB,60.95
152,2021-06-02,APOLLOTYRE,229.95
153,2021-06-02,RAYMOND,385.65
154,2021-06-02,INDUSINDBK,1028.00
155,2021-06-02,SUZLON,6.45
156,2021-06-02,CRISIL,2000.15
157,2021-06-02,IOB,17.50
158,2021-06-02,BALKRISIND,2229.40
159,2021-06-02,KAJARIACER,968.40


Feature Engineering: Adding variables in Ichimoku Calls dataset for Analysis 

In [34]:
c_swing_high = []
c_swing_low = []
call_activated = []
stop_loss_triggered = []
stoploss_trigr_dt = []
risk = []
target_1_ls = []
target_1point5_ls = []
target_2_ls = []
target_1_achieved_ls = []
target_1_dt_ls = []
target_1pt5_achieved_ls = []
target_1pt5_dt_ls = []
target_2_achieved_ls = []
target_2_dt_ls = []
as_on_date = []
as_on_date_closing = []


for i in np.arange(len(calls_j)):
    call_d = calls_j.iloc[i]['TIMESTAMP']
    sym = calls_j.iloc[i]['SYMBOL']
    temp = df_swing.loc[(df_swing['TIMESTAMP']>call_d) & (df_swing['SYMBOL']==sym)]

    # To consider the swing high that was just before the date on which the call was generated
    swing_high = df_swing.loc[(df_swing['TIMESTAMP']<call_d) & (df_swing['SYMBOL']==sym)][-2:-1]['swing_high'].tolist()[0]
    swing_low = df_swing.loc[(df_swing['TIMESTAMP']<call_d) & (df_swing['SYMBOL']==sym)][-2:-1]['swing_low'].tolist()[0]


    if (swing_high) >= (temp[:2]['LOW'].min()):
        call_activated.append('Yes')
    else:
        call_activated.append('No')
    if (swing_low >= (temp['LOW'].min())):
        stop_loss_triggered.append('Yes')
        stoploss_trigr_dt.append(temp.loc[temp['LOW']<=swing_low][0:1]['TIMESTAMP'].tolist()[0])
    else:
        stop_loss_triggered.append('No')
        stoploss_trigr_dt.append('')
    
    temp_risk = swing_high - swing_low
    
    temp_target_1 = temp_risk + swing_high
    temp_target_1pt5 = (temp_risk * 1.5) + swing_high
    temp_target_2 =  (temp_risk * 2) + swing_high

    c_swing_high.append(swing_high)
    c_swing_low.append(swing_low)
    risk.append(temp_risk)
    target_1_ls.append(temp_target_1)
    target_1point5_ls.append(temp_target_1pt5)
    target_2_ls.append(temp_target_2)

    if (temp['HIGH'].max() >= temp_target_1):
        target_1_achieved_ls.append('Yes')
        target_1_dt_ls.append(temp.loc[temp['HIGH']>=temp_target_1][0:1]['TIMESTAMP'].tolist()[0])
    else:
        target_1_achieved_ls.append('No')
        target_1_dt_ls.append('')

    if (temp['HIGH'].max() >= temp_target_1pt5):
        target_1pt5_achieved_ls.append('Yes')
        target_1pt5_dt_ls.append(temp.loc[temp['HIGH']>=temp_target_1pt5][0:1]['TIMESTAMP'].tolist()[0])
    else:
        target_1pt5_achieved_ls.append('No')
        target_1pt5_dt_ls.append('')

    if (temp['HIGH'].max() >= temp_target_2):
        target_2_achieved_ls.append('Yes')
        target_2_dt_ls.append(temp.loc[temp['HIGH']>=temp_target_2][0:1]['TIMESTAMP'].tolist()[0])
    else:
        target_2_achieved_ls.append('No')
        target_2_dt_ls.append('')

    as_on_date.append(df_swing.loc[df_swing['SYMBOL']==sym][-1:]['TIMESTAMP'].tolist()[0])
    as_on_date_closing.append(df_swing.loc[df_swing['SYMBOL']==sym][-1:]['CLOSE'].tolist()[0])

calls_j['Swing_H'] = pd.Series(c_swing_high)
calls_j['Swing_L'] = pd.Series(c_swing_low)
calls_j['Call_activated'] = pd.Series(call_activated)
calls_j['SL_triggered'] = pd.Series(stop_loss_triggered)
calls_j['SL_trig_dt'] = pd.Series(stoploss_trigr_dt)
calls_j['Risk'] = pd.Series(risk)
calls_j['LotSize_per_Lakh'] = round(1000/calls_j['Risk']).astype(int)

calls_j['T1'] = pd.Series(target_1_ls)
calls_j['T1_achieved'] = pd.Series(target_1_achieved_ls)
calls_j['T1_dt'] = pd.Series(target_1_dt_ls)

calls_j['T1.5'] = pd.Series(target_1point5_ls)
calls_j['T1.5_achieved'] = pd.Series(target_1pt5_achieved_ls)
calls_j['T1.5_dt'] = pd.Series(target_1pt5_dt_ls)

calls_j['T2'] = pd.Series(target_2_ls)
calls_j['T2_achieved'] = pd.Series(target_2_achieved_ls)
calls_j['T2_dt'] = pd.Series(target_2_dt_ls)

calls_j['As_on_dt'] = pd.Series(as_on_date)
calls_j['As_on_dt_closing'] = pd.Series(as_on_date_closing)

# sorting the dataframe in descending order with respect to 'TIMESTAMP' column
calls_j.sort_values(by=['TIMESTAMP'], ascending=False, inplace=True)

# Managing DateTime columns to display only date
calls_j['SL_trig_dt'] = calls_j['SL_trig_dt'].astype('datetime64[ns]')
dt_columns = ['TIMESTAMP', 'SL_trig_dt','T1_dt', 'T1.5_dt', 'T2_dt', 'As_on_dt']
for i in dt_columns:
    calls_j[i] = calls_j[i].dt.date


In [35]:
calls_j.head()

,TIMESTAMP,SYMBOL,CLOSE,Swing_H,Swing_L,Call_activated,SL_triggered,SL_trig_dt,Risk,LotSize_per_Lakh,T1,T1_achieved,T1_dt,T1.5,T1.5_achieved,T1.5_dt,T2,T2_achieved,T2_dt,As_on_dt,As_on_dt_closing
161,2021-06-02,VOLTAS,1034.40,1033.90,975.10,No,No,NaT,58.80,17,1092.70,No,NaT,1122.100,No,NaT,1151.50,No,NaT,2021-06-02,1034.40
156,2021-06-02,CRISIL,2000.15,1972.05,1894.05,No,No,NaT,78.00,13,2050.05,No,NaT,2089.050,No,NaT,2128.05,No,NaT,2021-06-02,2000.15
151,2021-06-02,IDFCFIRSTB,60.95,59.50,57.15,No,No,NaT,2.35,426,61.85,No,NaT,63.025,No,NaT,64.20,No,NaT,2021-06-02,60.95
152,2021-06-02,APOLLOTYRE,229.95,228.70,204.55,No,No,NaT,24.15,41,252.85,No,NaT,264.925,No,NaT,277.00,No,NaT,2021-06-02,229.95
154,2021-06-02,INDUSINDBK,1028.00,1022.75,888.10,No,No,NaT,134.65,7,1157.40,No,NaT,1224.725,No,NaT,1292.05,No,NaT,2021-06-02,1028.00


In [36]:
# calls_j.to_csv('drive/MyDrive/trading/WatchList_Calls/Ichimoku/ichimoku_calls_analysis.csv', index=False, encoding = 'utf-8')
calls_j.to_excel('drive/MyDrive/trading/WatchList_Calls/Ichimoku/ichimoku_calls_analysis.xlsx', index=False, encoding = 'utf-8')

Analysis for RSI Calls

In [37]:
r_calls = rsi_calls.copy()
calls_j = pd.merge(r_calls, df_swing, on =['SYMBOL','TIMESTAMP'], how ='left')[['TIMESTAMP','SYMBOL','CLOSE']]
calls_j[:10]

,TIMESTAMP,SYMBOL,CLOSE
0,2021-04-22,SUVENPHAR,523.15
1,2021-04-23,TTKPRESTIG,7671.45
2,2021-04-27,SUDARSCHEM,645.35
3,2021-04-27,SUPREMEIND,2050.35
4,2021-04-28,AEGISCHEM,313.35
5,2021-04-28,BAJAJFINSV,10489.30
6,2021-04-28,MFSL,919.50
7,2021-04-29,IDFC,51.80
8,2021-04-30,TATACOFFEE,131.00
9,2021-04-30,BASF,2228.75


In [38]:
c_swing_high = []
c_swing_low = []
call_activated = []
stop_loss_triggered = []
stoploss_trigr_dt = []
risk = []
target_1_ls = []
target_1point5_ls = []
target_2_ls = []
target_1_achieved_ls = []
target_1_dt_ls = []
target_1pt5_achieved_ls = []
target_1pt5_dt_ls = []
target_2_achieved_ls = []
target_2_dt_ls = []
as_on_date = []
as_on_date_closing = []


for i in np.arange(len(calls_j)):
    call_d = calls_j.iloc[i]['TIMESTAMP']
    sym = calls_j.iloc[i]['SYMBOL']
    temp = df_swing.loc[(df_swing['TIMESTAMP']>call_d) & (df_swing['SYMBOL']==sym)]

    # To consider the swing high that was just before the date on which the call was generated
    swing_high = df_swing.loc[(df_swing['TIMESTAMP']<call_d) & (df_swing['SYMBOL']==sym)][-2:-1]['swing_high'].tolist()[0]
    swing_low = df_swing.loc[(df_swing['TIMESTAMP']<call_d) & (df_swing['SYMBOL']==sym)][-2:-1]['swing_low'].tolist()[0]


    if (swing_high) >= (temp[:2]['LOW'].min()):
        call_activated.append('Yes')
    else:
        call_activated.append('No')
    if (swing_low >= (temp['LOW'].min())):
        stop_loss_triggered.append('Yes')
        stoploss_trigr_dt.append(temp.loc[temp['LOW']<=swing_low][0:1]['TIMESTAMP'].tolist()[0])
    else:
        stop_loss_triggered.append('No')
        stoploss_trigr_dt.append('')
    
    temp_risk = swing_high - swing_low
    
    temp_target_1 = temp_risk + swing_high
    temp_target_1pt5 = (temp_risk * 1.5) + swing_high
    temp_target_2 =  (temp_risk * 2) + swing_high

    c_swing_high.append(swing_high)
    c_swing_low.append(swing_low)
    risk.append(temp_risk)
    target_1_ls.append(temp_target_1)
    target_1point5_ls.append(temp_target_1pt5)
    target_2_ls.append(temp_target_2)

    if (temp['HIGH'].max() >= temp_target_1):
        target_1_achieved_ls.append('Yes')
        target_1_dt_ls.append(temp.loc[temp['HIGH']>=temp_target_1][0:1]['TIMESTAMP'].tolist()[0])
    else:
        target_1_achieved_ls.append('No')
        target_1_dt_ls.append('')

    if (temp['HIGH'].max() >= temp_target_1pt5):
        target_1pt5_achieved_ls.append('Yes')
        target_1pt5_dt_ls.append(temp.loc[temp['HIGH']>=temp_target_1pt5][0:1]['TIMESTAMP'].tolist()[0])
    else:
        target_1pt5_achieved_ls.append('No')
        target_1pt5_dt_ls.append('')

    if (temp['HIGH'].max() >= temp_target_2):
        target_2_achieved_ls.append('Yes')
        target_2_dt_ls.append(temp.loc[temp['HIGH']>=temp_target_2][0:1]['TIMESTAMP'].tolist()[0])
    else:
        target_2_achieved_ls.append('No')
        target_2_dt_ls.append('')

    as_on_date.append(df_swing.loc[df_swing['SYMBOL']==sym][-1:]['TIMESTAMP'].tolist()[0])
    as_on_date_closing.append(df_swing.loc[df_swing['SYMBOL']==sym][-1:]['CLOSE'].tolist()[0])

calls_j['Swing_H'] = pd.Series(c_swing_high)
calls_j['Swing_L'] = pd.Series(c_swing_low)
calls_j['Call_activated'] = pd.Series(call_activated)
calls_j['SL_triggered'] = pd.Series(stop_loss_triggered)
calls_j['SL_trig_dt'] = pd.Series(stoploss_trigr_dt)
calls_j['Risk'] = pd.Series(risk)
calls_j['LotSize_per_Lakh'] = round(1000/calls_j['Risk']).astype(int)

calls_j['T1'] = pd.Series(target_1_ls)
calls_j['T1_achieved'] = pd.Series(target_1_achieved_ls)
calls_j['T1_dt'] = pd.Series(target_1_dt_ls)

calls_j['T1.5'] = pd.Series(target_1point5_ls)
calls_j['T1.5_achieved'] = pd.Series(target_1pt5_achieved_ls)
calls_j['T1.5_dt'] = pd.Series(target_1pt5_dt_ls)

calls_j['T2'] = pd.Series(target_2_ls)
calls_j['T2_achieved'] = pd.Series(target_2_achieved_ls)
calls_j['T2_dt'] = pd.Series(target_2_dt_ls)

calls_j['As_on_dt'] = pd.Series(as_on_date)
calls_j['As_on_dt_closing'] = pd.Series(as_on_date_closing)

# sorting the dataframe in descending order with respect to 'TIMESTAMP' column
calls_j.sort_values(by=['TIMESTAMP'], ascending=False, inplace=True)

# Managing DateTime columns to display only date
calls_j['SL_trig_dt'] = calls_j['SL_trig_dt'].astype('datetime64[ns]')
dt_columns = ['TIMESTAMP', 'SL_trig_dt','T1_dt', 'T1.5_dt', 'T2_dt', 'As_on_dt']
for i in dt_columns:
    calls_j[i] = calls_j[i].dt.date

In [39]:
calls_j.head(10)

,TIMESTAMP,SYMBOL,CLOSE,Swing_H,Swing_L,Call_activated,SL_triggered,SL_trig_dt,Risk,LotSize_per_Lakh,T1,T1_achieved,T1_dt,T1.5,T1.5_achieved,T1.5_dt,T2,T2_achieved,T2_dt,As_on_dt,As_on_dt_closing
60,2021-06-02,VOLTAS,1034.40,1033.90,975.10,No,No,NaT,58.80,17,1092.70,No,NaT,1122.100,No,NaT,1151.50,No,NaT,2021-06-02,1034.40
59,2021-06-01,ADANIPORTS,798.20,779.90,749.45,No,No,NaT,30.45,33,810.35,Yes,2021-06-02,825.575,No,NaT,840.80,No,NaT,2021-06-02,811.50
58,2021-05-31,RATNAMANI,1975.70,1960.00,1891.10,Yes,No,NaT,68.90,15,2028.90,No,NaT,2063.350,No,NaT,2097.80,No,NaT,2021-06-02,1944.75
57,2021-05-31,NAM-INDIA,370.50,357.50,347.05,No,No,NaT,10.45,96,367.95,Yes,2021-06-01,373.175,Yes,2021-06-01,378.40,No,NaT,2021-06-02,372.25
56,2021-05-31,THYROCARE,1064.15,1049.00,993.25,Yes,No,NaT,55.75,18,1104.75,No,NaT,1132.625,No,NaT,1160.50,No,NaT,2021-06-02,1056.30
55,2021-05-31,LALPATHLAB,2956.15,2898.25,2740.00,Yes,No,NaT,158.25,6,3056.50,No,NaT,3135.625,No,NaT,3214.75,No,NaT,2021-06-02,2909.35
54,2021-05-28,METROPOLIS,2524.15,2390.00,2261.00,No,No,NaT,129.00,8,2519.00,Yes,2021-05-31,2583.500,Yes,2021-05-31,2648.00,Yes,2021-05-31,2021-06-02,2470.50
53,2021-05-27,BAJAJCON,292.25,288.95,270.35,Yes,No,NaT,18.60,54,307.55,No,NaT,316.850,No,NaT,326.15,No,NaT,2021-06-02,288.85
52,2021-05-26,INFY,1397.25,1373.05,1311.30,No,No,NaT,61.75,16,1434.80,No,NaT,1465.675,No,NaT,1496.55,No,NaT,2021-06-02,1378.65
51,2021-05-26,TEAMLEASE,3593.45,3415.45,3200.00,No,No,NaT,215.45,5,3630.90,Yes,2021-05-27,3738.625,No,NaT,3846.35,No,NaT,2021-06-02,3455.55


In [40]:
# calls_j.to_csv('drive/MyDrive/trading/WatchList_Calls/RSI/rsi_calls_analysis.csv', index=False, encoding = 'utf-8')
calls_j.to_excel('drive/MyDrive/trading/WatchList_Calls/RSI/rsi_calls_analysis.xlsx', index=False, encoding = 'utf-8')

In [41]:
print ("End of File. Thanks")

End of File. Thanks
